In [ ]:
#!pip install pandas
#!pip install yfinance
#!pip install matplotlib

import pandas as pd
import os
from datetime import datetime
import yfinance as yf
import Ab as ab
import datetime as dt

sd = dt.datetime(2000,1,1)
ed = dt.datetime(2023,11,17)

#read earning collection date from ./EarningCollectionDate.csv
earning_collection_date = pd.read_csv('./EarningCollectionDate.csv')
#column 'ticker', 'earning performance'
#convert 'ticker' to upper case
earning_collection_date['ticker'] = earning_collection_date['ticker'].str.upper()
#set the index to 'ticker'
earning_collection_date.set_index('ticker', inplace=True)
#if 'same' in 'earning performance' save 0 to 'performance date'
#if 'next' or 'next day' in 'earning performance' save 1 to 'performance date'
earning_collection_date['performance date'] = earning_collection_date['earning performance'].apply(lambda x: 0 if 'same' in x else 1)

#convert earning_collection_date to dictionary
earning_collection_date_dict = earning_collection_date.to_dict()['performance date']
print(earning_collection_date_dict)

# Define the directory path
directory = './Data-Cecile/'
output = './Processed_Data-Cecile/'

# Get a list of all the files in the directory
files = os.listdir(directory)
all_data = pd.DataFrame()
# Iterate over each file in the directory
for file in files:
    stock_name = file.split('.')[0]

    #convert stock_name to upper case
    stock_name = stock_name.upper()
    print(stock_name)
    earning_data = pd.read_csv(os.path.join(directory, file))

    earning_data.rename(columns={'earning date':'Earnings Date'}, inplace=True)
    #rename 'estimated eps':'EPS estimate'
    earning_data.rename(columns={'estimated eps ':'EPS estimate'}, inplace=True)
    #rename 'reported earning':'Reported EPS'
    earning_data.rename(columns={'reported earning':'Reported EPS'}, inplace=True)
    #rename 'reported eps':'Reported EPS'
    earning_data.rename(columns={'reported eps ':'Reported EPS'}, inplace=True)
    #rename 'earning surprise':'Surprise(%)'
    earning_data.rename(columns={'earning surprise':'Surprise(%)'}, inplace=True)
    #rename 'eps chg %: 'eps change%'
    earning_data.rename(columns={'eps chg %':'EPS change%'}, inplace=True)
    #rename 'eps change%':'EPS change%'
    earning_data.rename(columns={'eps change%':'EPS change%'}, inplace=True)
    earning_data.rename(columns={'eps change %':'EPS change%'}, inplace=True)
    #rename ' EPS estimate':'EPS estimate'
    earning_data.rename(columns={' EPS estimate':'EPS estimate'}, inplace=True)
    #'EPS Estimate'	'Reported EPS'	'Surprise(%)'

    # Convert the date to datetime and check the conversion
    earning_data['Earnings Date'] = pd.to_datetime(earning_data['Earnings Date'], utc=True)
    #lable the earning date as 1
    earning_data['Earning'] = 1
    # Remove the time from the date
    earning_data['Earnings Date'] = earning_data['Earnings Date'].dt.date
    # just inc one day to get the next day for the 'Date'

    #if the earning date is not the same day as the earning collection date, add one day to the earning date
    if(earning_collection_date_dict[stock_name] == 1):
        earning_data['Date'] = earning_data['Earnings Date'] + pd.DateOffset(1)
    else:
        earning_data['Date'] = earning_data['Earnings Date']
    # Set the 'Date' column as the index
    earning_data.set_index('Date', inplace=True)

    stock = ab.StockData(stock_name)
    stock.get_data_from_yfinance(stock_name, sd, ed)

    # Merge the earning data with the stock data
    stock.data = pd.merge(stock.data, earning_data, how='left', left_index=True, right_index=True)
    #plot the stock data and lable the earning date
    #stock.data.plot(y=['Close'], figsize=(16,1.5), sharey=True, color=['blue'],linewidth=0.7 ,title=stock_name)

    #is it a gap up day？
    stock.data['Gap Up'] = stock.data['Low'] > stock.data['High'].shift(1)
    stock.data['Price Change%'] = stock.data['Close'].pct_change(periods=1)
    stock.data['Gap%[(Low-Last Day High)/(Last Day High)]'] = (stock.data['Low'] - stock.data['High'].shift(1))/stock.data['High'].shift(1)
    stock.data['Spread%[(High - Low)/Low]'] = (stock.data['High'] - stock.data['Low'])/stock.data['Low']

    #stock.get_sma('Close', 5, 'SMA5')
    stock.get_sma('Close', 10, 'SMA10')
    stock.get_sma('Close', 21, 'SMA21')
    stock.get_sma('Close', 50, 'SMA50')
    stock.get_sma('Close', 150, 'SMA150')
    stock.get_sma('Close', 200, 'SMA200')
    stock.get_sma('Volume', 50, 'VOL50MA')
    stock.get_close_range(high='High', low='Low', close='Close', range='Close Range')

    #stock.get_to_percentage('Close', 'SMA5', 'TO_SMA5')
    stock.get_to_percentage('Close', 'SMA10', 'TO_SMA10')
    stock.get_to_percentage('Close', 'SMA21', 'TO_SMA21')
    stock.get_to_percentage('Close', 'SMA50', 'TO_SMA50')
    stock.get_to_percentage('Close', 'SMA150', 'TO_SMA150')
    stock.get_to_percentage('Close', 'SMA200', 'TO_SMA200')
    stock.get_to_percentage('Volume', 'VOL50MA', 'TO_VOL50MA')
    #get the stock gain percentage after earning date in 1w 2w 4w 8w
    stock.data['Performanc 1W'] = -stock.data['Close'].pct_change(periods=-5)
    stock.data['Performanc 2W'] = -stock.data['Close'].pct_change(periods=-10)
    stock.data['Performanc 4W'] = -stock.data['Close'].pct_change(periods=-20)
    stock.data['Performanc 5W'] = -stock.data['Close'].pct_change(periods=-25)
    stock.data['Performanc 8W'] = -stock.data['Close'].pct_change(periods=-40)
    stock.data['Performanc 13W'] = -stock.data['Close'].pct_change(periods=-(13*5))

    #backfill After_Earning column with 1 after earning date==1
    stock.data['After Earning'] = stock.data['Earning'].ffill(limit=13*5)
    #'Earning_Low' populate the earning date low price to the next 40 days

    #check if the stock price is higher than earning date low
    #stock.data['Break_Low'] = stock.data['Close'] < stock.data['Earning_Low']


    days_counter = 0
    for index, row in stock.data.iterrows():
        # Check if Earning equals 1
        if row['Earning'] == 1:
            # Record the Low value and reset the counter
            low_value_on_earning = row['Low']
            days_counter = 40

        # If we are in the 40-day period after an Earning == 1 event
        if days_counter > 0:
            stock.data.at[index, 'Earning Low'] = low_value_on_earning
            # Decrease the counter
            days_counter -= 1


    stock.data['Undercut Earning Low'] = stock.data['Close'] < stock.data['Earning Low']
    #under cut low 5%
    stock.data['Undercut Earning Close 5%'] = stock.data['Close'] < stock.data['Earning Low'] * 0.95
    #8% loss from gap up closing price
    stock.data['Undercut Earning Low 8%'] = stock.data['Close'] < stock.data['Earning Low'] * 0.92


    #add up the undercut earning column then convert to boolean
    stock.data['Undercut Earning 1W'] = stock.data['Undercut Earning Low'].shift(-5).rolling(5).sum()
    stock.data['Undercut Earning 2W'] = stock.data['Undercut Earning Low'].shift(-10).rolling(10).sum()
    stock.data['Undercut Earning 4W'] = stock.data['Undercut Earning Low'].shift(-20).rolling(20).sum()
    stock.data['Undercut Earning 8W'] = stock.data['Undercut Earning Low'].shift(-40).rolling(40).sum()

    stock.data['Undercut Earning Close 5% 1W'] = stock.data['Undercut Earning Close 5%'].shift(-5).rolling(5).sum()
    stock.data['Undercut Earning Close 5% 2W'] = stock.data['Undercut Earning Close 5%'].shift(-10).rolling(10).sum()
    stock.data['Undercut Earning Close 5% 4W'] = stock.data['Undercut Earning Close 5%'].shift(-20).rolling(20).sum()
    stock.data['Undercut Earning Close 5% 8W'] = stock.data['Undercut Earning Close 5%'].shift(-40).rolling(40).sum()

    stock.data['Undercut Earning Low 8% 1W'] = stock.data['Undercut Earning Low 8%'].shift(-5).rolling(5).sum()
    stock.data['Undercut Earning Low 8% 2W'] = stock.data['Undercut Earning Low 8%'].shift(-10).rolling(10).sum()
    stock.data['Undercut Earning Low 8% 4W'] = stock.data['Undercut Earning Low 8%'].shift(-20).rolling(20).sum()
    stock.data['Undercut Earning Low 8% 8W'] = stock.data['Undercut Earning Low 8%'].shift(-40).rolling(40).sum()
    #stock.data['EPS%'] = stock.data['EPS'].pct_change(periods=1)
    #stock.data['PRICE%']
    #stock.data['VOL_TO50MA']
    stock.data['SUCCESS GAP UP'] = ~(stock.data['Undercut Earning 8W'] > 0) & ~(stock.data['Undercut Earning Close 5% 8W'] > 0)
    #drop Open High Low Close Volume Close Earning 'After Earning' 'Undercut Earning' SMA10 SMA21 SMA50 SMA150 SMA200 VOL50MA
    stock.data.drop(columns=['Adj Close', 'Volume', 'After Earning', 'Earning Low'], inplace=True)
    stock.data.drop(columns=['SMA10', 'SMA21', 'SMA50', 'SMA150', 'SMA200', 'VOL50MA', 'Undercut Earning Low', 'Undercut Earning Close 5%', 'Undercut Earning Low 8%',], inplace=True)

    stock.data = stock.data[(stock.data['Earning'] == 1)]
    #stock.data = stock.data[(stock.data['Earning'] == 1) | (stock.data['After Earning'] == 1)]

    stock.data.drop(columns=['Earning'], inplace=True)
    stock.data['ticker'] = stock_name
    #only keep 3 digits after decimal point
    stock.data = stock.data.round(3)
    stock.data['Close Range'] = stock.data['Close Range'].round(2)
    stock.data.to_csv(output+stock_name+'.csv', index=True)
    #store stock.data to all_data
    all_data = pd.concat([all_data, stock.data])


all_data.to_csv(output + 'All_Data.csv', index=True)





